## Reading in packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import h5py
import copy
import datetime
import ta
import pathlib
import shutil
import tempfile
import vaex
from IPython import display
from IPython.display import clear_output
import pyodbc

# Tensorflow related
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import regularizers
import tensorflow.compat.v2.feature_column as fc

#!pip install -q git+https://github.com/tensorflow/docs

import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

print(tf.__version__)
logdir = pathlib.Path(tempfile.mkdtemp())/"tensorboard_logs"
shutil.rmtree(logdir, ignore_errors=True)
print(logdir)

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, roc_auc_score, f1_score, log_loss


# Models
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.exceptions import ConvergenceWarning 
from sklearn import ensemble
# ConvergenceWarning('ignore')
# Do you wanna see?
verbose = True

import sys
sys.path.append('../')
#sys.path.append('...../')

from utils.data_extraction import load_data_final,load_data_and_save
from utils.data_cleaning import HFDataCleaning
from utils.generate_features import candleCreateNP_vect_final,\
                                    generateFeatures_final,\
                                    generateFeatures_multi_final

from utils.preprocessing_features_and_labels import extract_labels,\
                                                    align_features_and_labels,\
                                                    pre_processing_initial,\
                                                    pre_processing_extended,\
                                                    pre_processing_final,\
                                                    extract_labels_multi_final,\
                                                    align_features_and_labels_multi_final,\
                                                    align_features_and_labels_multi_v5

from utils.models import make_input_fn
from utils.models import performanceTesting,scoreFunction
from utils.plotting import plot_confusion_matrix

2.2.0
C:\Users\PC\AppData\Local\Temp\tmps2yw09mt\tensorboard_logs


## Extracting data

In [2]:
# Do we extract new data or read in?
readIn = True
# run load_data()
if readIn:
    
    # Listing the data files 
    path = '../../../Google Drev/Thesis/Data/TAQ/AggregatedTAQ'
#     path = 'F:/AggregatedTAQ/round3'
    datafiles = os.listdir(path)
    content = np.concatenate([['\n\n'],[str(j)+': '+i+'\n' for j,i in enumerate(datafiles) if 'csv' in i],['\n\n']])
    
    # Asking for user input
    file = input('Which one do you want to load? %s'%''.join(content))
    if int(file) <= 2:
        data = pd.read_csv(path + '/' + datafiles[int(file)],
                           header = None,
                           names=['open','high','low','close',
                                  'spread_open','spread_high','spread_low','spread_close',
                                  'bidsize_open','bidsize_high','bidsize_low','bidsize_close',
                                  'ofrsize_open','ofrsize_high','ofrsize_low','ofrsize_close',
                                  'Ticker'])
        # Using the choice of the user to determine the correct market file
        key = re.split('[_.]',datafiles[int(file)])[-2]
        marketDataFile = [file for file in os.listdir(path+'/round5_market_tickers') if key in file]

        # Reading in the market data
        tempData = pd.read_csv(path+'/round5_market_tickers/'+marketDataFile[0]
                               ,header = None
                               ,names=['open','high','low','close',
                                      'spread_open','spread_high','spread_low','spread_close',
                                      'bidsize_open','bidsize_high','bidsize_low','bidsize_close',
                                      'ofrsize_open','ofrsize_high','ofrsize_low','ofrsize_close',
                                      'Ticker'])
        # Adding the market data to the ticker data
        data = pd.concat([data,tempData],axis=0)
    else:
        data = pd.read_csv(path + '/' + datafiles[int(file)],
                           header = 0,
                           index_col=[0,1]
#                            names=['open','high','low','close',
#                                   'spread_open','spread_high','spread_low','spread_close',
#                                   'bidsize_open','bidsize_high','bidsize_low','bidsize_close',
#                                   'ofrsize_open','ofrsize_high','ofrsize_low','ofrsize_close',
#                                   'Ticker']
                          )
    
    # Lower casing all column names
#     data.columns = data.columns.str.lower()
else:
    
    # print(os.listdir())
    try:
        path = 'a:/taqhdf5'  #'a:/taqhdf5'
        os.listdir(path)
    except:
        path = 't:/taqhdf5'  #'a:/taqhdf5'
        os.listdir(path)
        
    # Sample type
    data_sample = 'full' # or 'stable'
    # allFiles = os.listdir(path)
    # print(len(allFiles), allFiles[:5], allFiles[-5:])
    # print(allFiles[-10:])

    #dates = np.array(['2020040' + str(i) if i < 10 else '202004' + str(i) for i in np.arange(1,16)]).astype(int)
    dates = np.array(['20200501']).astype(int)#,'20200402','20200403','20200406','20200407'

    # Provide a list of tickers of interest
    
    tickers = sorted(['TSLA','FB'])#'MSFT'
    
    # Do we need data on trades, quotes or both?
    dataNeeded = 'quotes' # 'trades', 'quotes' or 'both'
    
    if dataNeeded == 'trades':
        tradeData = load_data_final(dates, tickers, dataNeeded, path, verbose)
    elif dataNeeded == 'quotes':
        quoteData = load_data_final(dates,
                                    tickers,
                                    dataNeeded,
                                    path,
                                    verbose,
                                    extract_candles = False,
                                    aggHorizon = 1,
                                    extra_features_from_quotes = None,
                                    data_sample = data_sample)
    elif dataNeeded == 'both':
        tradeData, quoteData = load_data_final(dates, tickers, dataNeeded, path, verbose)

# Reading in sector information
stockInfo = pd.read_csv('../utils/stockInfo_v1.csv',header=[0,1])
stockInfo.columns = ['ticker','sector','exchange','marketCap']

# Creating a table with stock information based on the tickers available in the data.
uniqueTickers = data.Ticker.unique()
stockTable = stockInfo[stockInfo.ticker.isin(uniqueTickers)]
stockTable.head(10)

Which one do you want to load? 

0: aggregateTAQ_May2020_10sec (1).csv
1: aggregateTAQ_May2020_30sec (1).csv
2: aggregateTAQ_May2020_60sec.csv
8: trueAggregateTAQ_60sec.csv


8


C:\Users\PC\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ticker,sector,exchange,marketCap
12,AAPL,Technology,NMS,1.578173e+12
20,ABBV,Healthcare,NYQ,1.742612e+11
34,ABT,Healthcare,NYQ,1.631410e+11
126,AEP,Utilities,NYQ,4.089551e+10
379,AMT,Real Estate,NYQ,1.171259e+11
428,APD,Basic Materials,NYQ,5.464395e+10
697,BA,Industrials,NYQ,1.020356e+11
699,BABA,Consumer Cyclical,NYQ,5.936536e+11
700,BAC,Financial Services,NYQ,2.020550e+11
870,BHP,Basic Materials,NYQ,1.258194e+11


In [3]:
files = os.listdir('../AzureML/Output_from_cloud')
files

['hyperparameters.txt', 'metrics.txt']

# Hyperparameters

In [4]:
with open('../AzureML/Output_from_cloud/'+files[0],'r') as file:
    content = file.readlines()

In [5]:
content[0]

'HD_81ace623-e09b-4393-8a5e-131ea8749a35_0\t{"activation-inner": "sigmoid", "activation-output": "linear", "batch-norm": 0, "batch-shuffle": 1, "batch-size": 3300, "dropout-ratio": 0.1, "feature-lags": 3, "featureset": 2, "first-layer-neurons": 32, "label-type": 0, "learning-rate": 0.01, "n-epochs": 150, "n-layers": 3, "nn-type": "ffnn", "pastobs-in-percentage": 1, "pre-processing": "stacked", "second-layer-neurons": 64}\n'

In [6]:
content[0]

'HD_81ace623-e09b-4393-8a5e-131ea8749a35_0\t{"activation-inner": "sigmoid", "activation-output": "linear", "batch-norm": 0, "batch-shuffle": 1, "batch-size": 3300, "dropout-ratio": 0.1, "feature-lags": 3, "featureset": 2, "first-layer-neurons": 32, "label-type": 0, "learning-rate": 0.01, "n-epochs": 150, "n-layers": 3, "nn-type": "ffnn", "pastobs-in-percentage": 1, "pre-processing": "stacked", "second-layer-neurons": 64}\n'

In [7]:
ids = re.split('[\t]',content[0])[0]
parameters = re.split('[\t]',content[0])[1]
ids,'\n',parameters

('HD_81ace623-e09b-4393-8a5e-131ea8749a35_0',
 '\n',
 '{"activation-inner": "sigmoid", "activation-output": "linear", "batch-norm": 0, "batch-shuffle": 1, "batch-size": 3300, "dropout-ratio": 0.1, "feature-lags": 3, "featureset": 2, "first-layer-neurons": 32, "label-type": 0, "learning-rate": 0.01, "n-epochs": 150, "n-layers": 3, "nn-type": "ffnn", "pastobs-in-percentage": 1, "pre-processing": "stacked", "second-layer-neurons": 64}\n')

In [8]:
test = [re.split(': ',i.strip()) for i in re.split(',',parameters.replace('{','').replace('}','').replace('\n','').replace('"',''))]
test = {i[0]:i[1] for i in test}

In [9]:
test

{'activation-inner': 'sigmoid',
 'activation-output': 'linear',
 'batch-norm': '0',
 'batch-shuffle': '1',
 'batch-size': '3300',
 'dropout-ratio': '0.1',
 'feature-lags': '3',
 'featureset': '2',
 'first-layer-neurons': '32',
 'label-type': '0',
 'learning-rate': '0.01',
 'n-epochs': '150',
 'n-layers': '3',
 'nn-type': 'ffnn',
 'pastobs-in-percentage': '1',
 'pre-processing': 'stacked',
 'second-layer-neurons': '64'}

In [10]:
pd.DataFrame(test,index=[ids])

,activation-inner,activation-output,batch-norm,batch-shuffle,batch-size,dropout-ratio,feature-lags,featureset,first-layer-neurons,label-type,learning-rate,n-epochs,n-layers,nn-type,pastobs-in-percentage,pre-processing,second-layer-neurons
HD_81ace623-e09b-4393-8a5e-131ea8749a35_0,sigmoid,linear,0,1,3300,0.1,3,2,32,0,0.01,150,3,ffnn,1,stacked,64


# Combining it all

In [11]:
all_ids = []
all_parameters = []

for a in np.arange(1000):
    
#     all_ids.append(re.split('[\t]',content[0])[0])
#     all_parameters.append(re.split('[\t]',content[a])[1])
    temp_parameters = re.split('[\t]',content[a])[1]
    
    test = [re.split(': ',i.strip()) for i in re.split(',',temp_parameters.replace('{','')\
                                                                       .replace('}','')\
                                                                       .replace('\n','')\
                                                                       .replace('"',''))]
    test = {i[0]:i[1] for i in test}
    
    if a == 0:
        parameters = pd.DataFrame(test,index=[re.split('[\t]',content[a])[0]])
    
    else:
        
        parameters.loc[re.split('[\t]',content[a])[0]] = pd.Series(test)

In [12]:
parameters

,activation-inner,activation-output,batch-norm,batch-shuffle,batch-size,dropout-ratio,feature-lags,featureset,first-layer-neurons,label-type,learning-rate,n-epochs,n-layers,nn-type,pastobs-in-percentage,pre-processing,second-layer-neurons
HD_81ace623-e09b-4393-8a5e-131ea8749a35_0,sigmoid,linear,0,1,3300,0.1,3,2,32,0,0.01,150,3,ffnn,1,stacked,64
HD_81ace623-e09b-4393-8a5e-131ea8749a35_1,sigmoid,linear,0,1,21450,0.1,1,1,64,1,0.1,150,3,ffnn,1,stacked,64
HD_81ace623-e09b-4393-8a5e-131ea8749a35_2,sigmoid,softmax,0,0,3300,0.4,0,2,32,4,0.1,150,2,ffnn,0,None,64
HD_81ace623-e09b-4393-8a5e-131ea8749a35_3,tanh,linear,0,0,10725,0.5,0,1,64,0,0.01,150,2,ffnn,0,quantgau,128
HD_81ace623-e09b-4393-8a5e-131ea8749a35_4,leakyrelu,linear,0,1,21450,0.4,1,2,128,0,0.001,150,2,lstm,1,None,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HD_81ace623-e09b-4393-8a5e-131ea8749a35_995,sigmoid,linear,1,0,21450,0.0,3,3,64,2,0.0001,150,4,ffnn,0,None,64
HD_81ace623-e09b-4393-8a5e-131ea8749a35_996,sigmoid,softmax,1,1,10725,0.0,3,0,64,1,0.01,150,4,ffnn,1,std,64
HD_81ace623-e09b-4393-8a5e-131ea8749a35_997,relu,linear,1,1,10725,0.5,3,2,64,3,0.0001,150,2,lstm,0,pow,64
HD_81ace623-e09b-4393-8a5e-131ea8749a35_998,sigmoid,softmax,0,0,21450,0.3,5,3,128,2,0.001,150,2,lstm,0,stacked,64


# Metrics

In [13]:
with open('../AzureML/Output_from_cloud/'+files[1],'r') as file:
    content = file.readlines()

In [14]:
content[0]

'HD_81ace623-e09b-4393-8a5e-131ea8749a35_0\tLoss\t[1.0985371210087256, 1.0985042562539795, 1.0984668008328402, 1.0984232064009172, 1.0983712081346395, 1.0983087202155337, 1.098232851799212, 1.0981401157654425, 1.0980258930792328, 1.0978860390193956, 1.0977150399842424, 1.097505593697485, 1.0972516788283244, 1.0969440557774708, 1.0965741972956924, 1.0961331910548722, 1.095607562429099, 1.0949885457505624, 1.0942660295634488, 1.0934271321532205, 1.0924700422115705, 1.09139323700996, 1.0902104344866534, 1.0889394297823072, 1.087614836411418, 1.0862864393542562, 1.0849873849105345, 1.0837727543480355, 1.0826659956522826, 1.081669953329736, 1.0807763167571531, 1.080011005505604, 1.0793186371872407, 1.0786993224472476, 1.078135087139871, 1.07761496324582, 1.0771327582286518, 1.0766734054564207, 1.076243416860824, 1.0758342227544289, 1.0754266440524285, 1.0750576965009042, 1.0746909418956372, 1.0743327297561822, 1.073974203879899, 1.0736468839217181, 1.0733284460878891, 1.073017609509388, 1.0

In [17]:
re.split('\t',content[0].replace('\n',''))

['HD_81ace623-e09b-4393-8a5e-131ea8749a35_0',
 'Loss',
 '[1.0985371210087256, 1.0985042562539795, 1.0984668008328402, 1.0984232064009172, 1.0983712081346395, 1.0983087202155337, 1.098232851799212, 1.0981401157654425, 1.0980258930792328, 1.0978860390193956, 1.0977150399842424, 1.097505593697485, 1.0972516788283244, 1.0969440557774708, 1.0965741972956924, 1.0961331910548722, 1.095607562429099, 1.0949885457505624, 1.0942660295634488, 1.0934271321532205, 1.0924700422115705, 1.09139323700996, 1.0902104344866534, 1.0889394297823072, 1.087614836411418, 1.0862864393542562, 1.0849873849105345, 1.0837727543480355, 1.0826659956522826, 1.081669953329736, 1.0807763167571531, 1.080011005505604, 1.0793186371872407, 1.0786993224472476, 1.078135087139871, 1.07761496324582, 1.0771327582286518, 1.0766734054564207, 1.076243416860824, 1.0758342227544289, 1.0754266440524285, 1.0750576965009042, 1.0746909418956372, 1.0743327297561822, 1.073974203879899, 1.0736468839217181, 1.0733284460878891, 1.0730176095093

In [40]:
[float(j.strip()) for j in re.split(',',re.split('\t',content[0].replace('\n',''))[2].replace('[','').replace(']',''))]

[1.0985371210087256,
 1.0985042562539795,
 1.0984668008328402,
 1.0984232064009172,
 1.0983712081346395,
 1.0983087202155337,
 1.098232851799212,
 1.0981401157654425,
 1.0980258930792328,
 1.0978860390193956,
 1.0977150399842424,
 1.097505593697485,
 1.0972516788283244,
 1.0969440557774708,
 1.0965741972956924,
 1.0961331910548722,
 1.095607562429099,
 1.0949885457505624,
 1.0942660295634488,
 1.0934271321532205,
 1.0924700422115705,
 1.09139323700996,
 1.0902104344866534,
 1.0889394297823072,
 1.087614836411418,
 1.0862864393542562,
 1.0849873849105345,
 1.0837727543480355,
 1.0826659956522826,
 1.081669953329736,
 1.0807763167571531,
 1.080011005505604,
 1.0793186371872407,
 1.0786993224472476,
 1.078135087139871,
 1.07761496324582,
 1.0771327582286518,
 1.0766734054564207,
 1.076243416860824,
 1.0758342227544289,
 1.0754266440524285,
 1.0750576965009042,
 1.0746909418956372,
 1.0743327297561822,
 1.073974203879899,
 1.0736468839217181,
 1.0733284460878891,
 1.073017609509388,
 1.072

In [42]:
test = np.zeros(150)
test1 = [float(j.strip()) for j in re.split(',',re.split('\t',content[0].replace('\n',''))[2].replace('[','').replace(']',''))]
test[0:len(test1)] = test1
test[len(test1):] = test1[-1]
test

array([1.09853712, 1.09850426, 1.0984668 , 1.09842321, 1.09837121,
       1.09830872, 1.09823285, 1.09814012, 1.09802589, 1.09788604,
       1.09771504, 1.09750559, 1.09725168, 1.09694406, 1.0965742 ,
       1.09613319, 1.09560756, 1.09498855, 1.09426603, 1.09342713,
       1.09247004, 1.09139324, 1.09021043, 1.08893943, 1.08761484,
       1.08628644, 1.08498738, 1.08377275, 1.082666  , 1.08166995,
       1.08077632, 1.08001101, 1.07931864, 1.07869932, 1.07813509,
       1.07761496, 1.07713276, 1.07667341, 1.07624342, 1.07583422,
       1.07542664, 1.0750577 , 1.07469094, 1.07433273, 1.0739742 ,
       1.07364688, 1.07332845, 1.07301761, 1.07270177, 1.0724003 ,
       1.07210017, 1.07182065, 1.07155601, 1.07129143, 1.07103606,
       1.07077294, 1.07052828, 1.07031799, 1.07008369, 1.06986291,
       1.06964009, 1.06942849, 1.06921837, 1.06903996, 1.06885742,
       1.06867783, 1.06851121, 1.06833709, 1.06817681, 1.06800568,
       1.0678564 , 1.06771191, 1.06755972, 1.06741042, 1.06727

In [22]:
metricLabels = []

for i in np.arange(18):
    
    metricLabels.append(re.split('\t',content[i].replace('\n',''))[1])
    
print(set(metricLabels),len(set(metricLabels)))

{'Train Accuracy', 'Final test loss', 'Loss', 'Accuracy', 'Train Loss', 'Train AUC', 'AUC', 'Final test accuracy', 'Final test AUC'} 9


In [44]:
finalMetrics

,metric,size


In [61]:
temp[0],temp[1]

('HD_81ace623-e09b-4393-8a5e-131ea8749a35_0', 'Loss')

In [63]:
[temp[0],temp[1]]

['HD_81ace623-e09b-4393-8a5e-131ea8749a35_0', 'Loss']

In [64]:
pd.MultiIndex.from_product([[temp[0]],[temp[1]]])

MultiIndex([('HD_81ace623-e09b-4393-8a5e-131ea8749a35_0', 'Loss')],
           )

In [98]:
t11 = []
t12 = []
t13 = []

t21 = []
t22 = []
t23 = []


for i in np.arange(len(content)):#
    
    temp = re.split('\t',content[i].replace('\n',''))
#     print(temp,'\n')
    if len(temp[2]) < 50:
        
#         finalMetrics.loc[pd.MultiIndex.from_product([[temp[0]],[temp[1]]])] = [float(temp[2])]
#         finalMetrics.loc[temp[0]] = [temp[1],float(temp[2])]

        t11.append(temp[0])
        t12.append(temp[1])
        t13.append(temp[2])

    else:
    
        container = np.zeros(155)
        temp1 = [float(j.strip()) if j.strip() !="'NaN'" else 0 for j in re.split(',',temp[2].replace('[','').replace(']',''))]
#         if len(temp1) > 150:
#             print(temp1)
        container[0:len(temp1)] = temp1
        container[len(temp1):] = temp1[-1]
#         print(container)
#         runningMetrics.loc[pd.MultiIndex.from_product([[temp[0]],[temp[1]]])] = container

        t21.append(temp[0])
        t22.append(temp[1])
        t23.append(container)
#         runningMetrics.loc[temp[0]] = np.concatenate([[temp[1]],container])
        
    
#     print(re.split('\t',content[i].replace('\n',''))[2],len(re.split('\t',content[i].replace('\n',''))[2]))

arrays = [t21,t22]
tuples = list(zip(*arrays))

runningMetrics = pd.DataFrame(np.array(t23),
                              index=pd.MultiIndex.from_tuples(tuples),
                              columns = [np.arange(155).astype(str)]
                             )
arrays = [t11,t12]
tuples = list(zip(*arrays))
finalMetrics = pd.DataFrame(t13,index = pd.MultiIndex.from_tuples(tuples),columns = ['size'])

In [99]:
# t11,t12,t13

In [100]:
# t21[0],t22[0],t23[0]

In [101]:
# np.array(t23).shape

In [102]:
finalMetrics

size
HD_81ace623-e09b-4393-8a5e-131ea8749a35_1   Final test loss      0.8833522492045336
                                            Final test AUC       0.7648658156394958
                                            Final test accuracy  0.6109746098518372
HD_81ace623-e09b-4393-8a5e-131ea8749a35_3   Final test loss      1.1133369887535414
                                            Final test AUC        0.640163779258728
...                                                                             ...
HD_81ace623-e09b-4393-8a5e-131ea8749a35_997 Final test AUC       0.7837895154953003
                                            Final test accuracy   0.602816104888916
HD_81ace623-e09b-4393-8a5e-131ea8749a35_998 Final test loss      1.3713646207894699
                                            Final test AUC       0.7241190671920776
                                            Final test accuracy  0.4148908853530884

[1170 rows x 1 columns]

In [103]:
runningMetrics

0  \
HD_81ace623-e09b-4393-8a5e-131ea8749a35_0   Loss            1.098537   
                                            Accuracy        0.354843   
                                            AUC             0.500120   
                                            Train Loss      1.098558   
                                            Train Accuracy  0.344181   
...                                                              ...   
HD_81ace623-e09b-4393-8a5e-131ea8749a35_999 Accuracy        0.337729   
                                            AUC             0.006679   
                                            Train Loss      5.383223   
                                            Train Accuracy  0.333057   
                                            Train AUC       0.030584   

                                                                   1  \
HD_81ace623-e09b-4393-8a5e-131ea8749a35_0   Loss            1.098504   
                                            Accuracy        0.358482   
                                            AUC             0.500277   
                                            Train Loss      1.098524   
                                            Train Accuracy  0.350251   
...                                                              ...   
HD_81ace623-e09b-4393-8a5e-131ea8749a35_999 Accuracy        0.337729   
                                            AUC             0.003153   
                                            Train Loss      5.355752   
                                            Train Accuracy  0.332281   
                                            Train AUC       0.004384   

                                                                   2  \
HD_81ace623-e09b-4393-8a5e-131ea8749a35_0   Loss            1.098467   
                                            Accuracy        0.362843   
                                            AUC             0.500545   
                                            Train Loss      1.098487   
                                            Train Accuracy  0.353537   
...                                                              ...   
HD_81ace623-e09b-4393-8a5e-131ea8749a35_999 Accuracy        0.337729   
                                            AUC             0.002064   
                                            Train Loss      5.355751   
                                            Train Accuracy  0.332281   
                                            Train AUC       0.002511   

                                                                   3  \
HD_81ace623-e09b-4393-8a5e-131ea8749a35_0   Loss            1.098423   
                                            Accuracy        0.365199   
                                            AUC             0.501018   
                                            Train Loss      1.098442   
                                            Train Accuracy  0.358348   
...                                                              ...   
HD_81ace623-e09b-4393-8a5e-131ea8749a35_999 Accuracy        0.337729   
                                            AUC             0.001534   
                                            Train Loss      5.355751   
                                            Train Accuracy  0.332281   
                                            Train AUC       0.001766   

                                                                   4  \
HD_81ace623-e09b-4393-8a5e-131ea8749a35_0   Loss            1.098371   
                                            Accuracy        0.366960   
                                            AUC             0.501737   
                                            Train Loss      1.098396   
                                            Train Accuracy  0.360740   
...                                                              ...   
HD_81ace623-e09b-4393-8a5e-131ea8749a35_999 Accuracy        0.337729   
                               

In [104]:
runningMetrics.index.get_level_values(1).unique()

Index(['Loss', 'Accuracy', 'AUC', 'Train Loss', 'Train Accuracy', 'Train AUC'], dtype='object')

In [105]:
finalMetrics.index.get_level_values(1).unique()

Index(['Final test loss', 'Final test AUC', 'Final test accuracy'], dtype='object')

In [106]:
runningMetrics[runningMetrics.index.get_level_values(1)=='Final test AUC']

,,0,1,2,3,4,5,6,7,8,9,...,145,146,147,148,149,150,151,152,153,154


In [107]:
finalMetrics[finalMetrics.index.get_level_values(1)=='Final test AUC']

,,size
HD_81ace623-e09b-4393-8a5e-131ea8749a35_1,Final test AUC,0.7648658156394958
HD_81ace623-e09b-4393-8a5e-131ea8749a35_3,Final test AUC,0.640163779258728
HD_81ace623-e09b-4393-8a5e-131ea8749a35_4,Final test AUC,0.6167277097702026
HD_81ace623-e09b-4393-8a5e-131ea8749a35_12,Final test AUC,0.7076375484466553
HD_81ace623-e09b-4393-8a5e-131ea8749a35_14,Final test AUC,0.6344101428985596
...,...,...
HD_81ace623-e09b-4393-8a5e-131ea8749a35_991,Final test AUC,0.044382814317941666
HD_81ace623-e09b-4393-8a5e-131ea8749a35_992,Final test AUC,0.4694923758506775
HD_81ace623-e09b-4393-8a5e-131ea8749a35_996,Final test AUC,0.453373521566391
HD_81ace623-e09b-4393-8a5e-131ea8749a35_997,Final test AUC,0.7837895154953003


In [109]:
runningMetrics[runningMetrics.index.get_level_values(1)=='AUC'].iloc[:,-1]

HD_81ace623-e09b-4393-8a5e-131ea8749a35_0    AUC    0.575838
HD_81ace623-e09b-4393-8a5e-131ea8749a35_1    AUC    0.764845
HD_81ace623-e09b-4393-8a5e-131ea8749a35_2    AUC    0.115722
HD_81ace623-e09b-4393-8a5e-131ea8749a35_3    AUC    0.640223
HD_81ace623-e09b-4393-8a5e-131ea8749a35_4    AUC    0.616729
                                                      ...   
HD_81ace623-e09b-4393-8a5e-131ea8749a35_995  AUC    0.000045
HD_81ace623-e09b-4393-8a5e-131ea8749a35_996  AUC    0.453591
HD_81ace623-e09b-4393-8a5e-131ea8749a35_997  AUC    0.783799
HD_81ace623-e09b-4393-8a5e-131ea8749a35_998  AUC    0.724110
HD_81ace623-e09b-4393-8a5e-131ea8749a35_999  AUC    0.000113
Name: (154,), Length: 1000, dtype: float64

In [126]:
# type(runningMetrics[runningMetrics.index.get_level_values(1)=='AUC'].reset_index()[['level_0','level_1','154']])
runningMetrics[runningMetrics.index.get_level_values(1)=='AUC'].reset_index()[['level_0','level_1','154']]

pandas.core.frame.DataFrame

In [112]:
runningMetrics[runningMetrics.index.get_level_values(1)=='AUC'].iloc[:,-1].reset_index()

,level_0,level_1,154
0,HD_81ace623-e09b-4393-8a5e-131ea8749a35_0,AUC,0.575838
1,HD_81ace623-e09b-4393-8a5e-131ea8749a35_1,AUC,0.764845
2,HD_81ace623-e09b-4393-8a5e-131ea8749a35_2,AUC,0.115722
3,HD_81ace623-e09b-4393-8a5e-131ea8749a35_3,AUC,0.640223
4,HD_81ace623-e09b-4393-8a5e-131ea8749a35_4,AUC,0.616729
...,...,...,...
995,HD_81ace623-e09b-4393-8a5e-131ea8749a35_995,AUC,0.000045
996,HD_81ace623-e09b-4393-8a5e-131ea8749a35_996,AUC,0.453591
997,HD_81ace623-e09b-4393-8a5e-131ea8749a35_997,AUC,0.783799
998,HD_81ace623-e09b-4393-8a5e-131ea8749a35_998,AUC,0.724110


In [111]:
parameters = parameters.reset_index()

In [113]:
parameters

,index,activation-inner,activation-output,batch-norm,batch-shuffle,batch-size,dropout-ratio,feature-lags,featureset,first-layer-neurons,label-type,learning-rate,n-epochs,n-layers,nn-type,pastobs-in-percentage,pre-processing,second-layer-neurons
0,HD_81ace623-e09b-4393-8a5e-131ea8749a35_0,sigmoid,linear,0,1,3300,0.1,3,2,32,0,0.01,150,3,ffnn,1,stacked,64
1,HD_81ace623-e09b-4393-8a5e-131ea8749a35_1,sigmoid,linear,0,1,21450,0.1,1,1,64,1,0.1,150,3,ffnn,1,stacked,64
2,HD_81ace623-e09b-4393-8a5e-131ea8749a35_2,sigmoid,softmax,0,0,3300,0.4,0,2,32,4,0.1,150,2,ffnn,0,None,64
3,HD_81ace623-e09b-4393-8a5e-131ea8749a35_3,tanh,linear,0,0,10725,0.5,0,1,64,0,0.01,150,2,ffnn,0,quantgau,128
4,HD_81ace623-e09b-4393-8a5e-131ea8749a35_4,leakyrelu,linear,0,1,21450,0.4,1,2,128,0,0.001,150,2,lstm,1,None,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,HD_81ace623-e09b-4393-8a5e-131ea8749a35_995,sigmoid,linear,1,0,21450,0.0,3,3,64,2,0.0001,150,4,ffnn,0,None,64
996,HD_81ace623-e09b-4393-8a5e-131ea8749a35_996,sigmoid,softmax,1,1,10725,0.0,3,0,64,1,0.01,150,4,ffnn,1,std,64
997,HD_81ace623-e09b-4393-8a5e-131ea8749a35_997,relu,linear,1,1,10725,0.5,3,2,64,3,0.0001,150,2,lstm,0,pow,64
998,HD_81ace623-e09b-4393-8a5e-131ea8749a35_998,sigmoid,softmax,0,0,21450,0.3,5,3,128,2,0.001,150,2,lstm,0,stacked,64


In [125]:
parameters.merge(runningMetrics[runningMetrics.index.get_level_values(1)=='AUC'].reset_index()[['level_0','level_1','154']],#runningMetrics[runningMetrics.index.get_level_values(1)=='AUC'].iloc[:,-1].reset_index(),
                 left_on='index',
                 right_on='level_0',
                 how='left')

TypeError: only integer scalar arrays can be converted to a scalar index

In [91]:
hej = np.concatenate([[[i,j] for i in np.arange(2)] for j in np.arange(4)])

In [92]:
hej

array([[0, 0],
       [1, 0],
       [0, 1],
       [1, 1],
       [0, 2],
       [1, 2],
       [0, 3],
       [1, 3]])

# reading in the market data (done automatically atm)

In [3]:
file

'8'

In [4]:
data

open     high      low    close  spread_open  spread_high  \
20200501 0    286.250  289.260  285.870  289.260         0.50         0.50   
         1    289.260  289.350  288.365  289.020         0.24         0.45   
         2    289.035  289.705  288.280  288.580         0.07         0.49   
         3    288.485  289.315  288.280  289.095         0.49         0.49   
         4    289.100  290.435  288.940  290.320         0.16         0.33   
...               ...      ...      ...      ...          ...          ...   
20200529 385   91.500   91.755   91.485   91.740         0.42         0.93   
         386   91.740   91.740   91.740   91.740         0.50         0.50   
         387   91.580   91.830   91.580   91.715         0.18         0.68   
         388   91.595   91.880   91.595   91.750         0.21         0.78   
         389   46.005   46.005   45.815   45.815        92.01        92.01   

              spread_low  spread_close  bidsize_open  bidsize_high  \
20200501 0          0.01          0.24           6.0          95.0   
         1          0.01          0.10           9.0          20.0   
         2          0.01          0.30           1.0          50.0   
         3          0.01          0.17           1.0          25.0   
         4          0.01          0.10          13.0          71.0   
...                  ...           ...           ...           ...   
20200529 385        0.39          0.50           5.0           5.0   
         386        0.50          0.50           5.0           5.0   
         387        0.18          0.45           5.0           5.0   
         388        0.21          0.52           5.0           5.0   
         389       91.63         91.63           0.0           0.0   

              bidsize_low  bidsize_close  ofrsize_open  ofrsize_high  \
20200501 0            1.0           10.0           1.0          85.0   
         1            1.0            1.0           4.0          56.0   
         2            1.0            1.0           1.0          13.0   
         3            1.0           16.0           1.0           8.0   
         4            1.0            1.0           1.0         236.0   
...                   ...            ...           ...           ...   
20200529 385          5.0            5.0           1.0           5.0   
         386          5.0            5.0           5.0           5.0   
         387          5.0            5.0           1.0           5.0   
         388          5.0            5.0           1.0           5.0   
         389          0.0            0.0           5.0           5.0   

              ofrsize_low  ofrsize_close Ticker      sector  
20200501 0            1.0            4.0   AAPL  Technology  
         1            1.0            1.0   AAPL  Technology  
         2            1.0            1.0   AAPL  Technology  
         3            1.0            1.0   AAPL  Technology  
         4            1.0            1.0   AAPL  Technology  
...                   ...            ...    ...         ...  
20200529 385          1.0            5.0   XNTK         NaN  
         386          5.0            5.0   XNTK         NaN  
         387          1.0            5.0   XNTK         NaN  
         388          1.0            5.0   XNTK         NaN  
         389          1.0            1.0   XNTK         NaN  

[546000 rows x 18 columns]

In [5]:
data

open     high      low    close  spread_open  spread_high  \
20200501 0    286.250  289.260  285.870  289.260         0.50         0.50   
         1    289.260  289.350  288.365  289.020         0.24         0.45   
         2    289.035  289.705  288.280  288.580         0.07         0.49   
         3    288.485  289.315  288.280  289.095         0.49         0.49   
         4    289.100  290.435  288.940  290.320         0.16         0.33   
...               ...      ...      ...      ...          ...          ...   
20200529 385   91.500   91.755   91.485   91.740         0.42         0.93   
         386   91.740   91.740   91.740   91.740         0.50         0.50   
         387   91.580   91.830   91.580   91.715         0.18         0.68   
         388   91.595   91.880   91.595   91.750         0.21         0.78   
         389   46.005   46.005   45.815   45.815        92.01        92.01   

              spread_low  spread_close  bidsize_open  bidsize_high  \
20200501 0          0.01          0.24           6.0          95.0   
         1          0.01          0.10           9.0          20.0   
         2          0.01          0.30           1.0          50.0   
         3          0.01          0.17           1.0          25.0   
         4          0.01          0.10          13.0          71.0   
...                  ...           ...           ...           ...   
20200529 385        0.39          0.50           5.0           5.0   
         386        0.50          0.50           5.0           5.0   
         387        0.18          0.45           5.0           5.0   
         388        0.21          0.52           5.0           5.0   
         389       91.63         91.63           0.0           0.0   

              bidsize_low  bidsize_close  ofrsize_open  ofrsize_high  \
20200501 0            1.0           10.0           1.0          85.0   
         1            1.0            1.0           4.0          56.0   
         2            1.0            1.0           1.0          13.0   
         3            1.0           16.0           1.0           8.0   
         4            1.0            1.0           1.0         236.0   
...                   ...            ...           ...           ...   
20200529 385          5.0            5.0           1.0           5.0   
         386          5.0            5.0           5.0           5.0   
         387          5.0            5.0           1.0           5.0   
         388          5.0            5.0           1.0           5.0   
         389          0.0            0.0           5.0           5.0   

              ofrsize_low  ofrsize_close Ticker      sector  
20200501 0            1.0            4.0   AAPL  Technology  
         1            1.0            1.0   AAPL  Technology  
         2            1.0            1.0   AAPL  Technology  
         3            1.0            1.0   AAPL  Technology  
         4            1.0            1.0   AAPL  Technology  
...                   ...            ...    ...         ...  
20200529 385          1.0            5.0   XNTK         NaN  
         386          5.0            5.0   XNTK         NaN  
         387          1.0            5.0   XNTK         NaN  
         388          1.0            5.0   XNTK         NaN  
         389          1.0            1.0   XNTK         NaN  

[546000 rows x 18 columns]

In [6]:
data.columns

Index(['open', 'high', 'low', 'close', 'spread_open', 'spread_high',
       'spread_low', 'spread_close', 'bidsize_open', 'bidsize_high',
       'bidsize_low', 'bidsize_close', 'ofrsize_open', 'ofrsize_high',
       'ofrsize_low', 'ofrsize_close', 'Ticker', 'sector'],
      dtype='object')

### Dropping ETFS and market indices

In [4]:
data.Ticker.unique()

array(['AAPL', 'ABBV', 'ABT', 'AEP', 'AMT', 'APD', 'BA', 'BABA', 'BAC',
       'BHP', 'BP', 'CCI', 'CHL', 'COST', 'CSGP', 'D', 'DIS', 'ECL',
       'ENB', 'EXC', 'FB', 'FMX', 'GOOG', 'IDU', 'INTC', 'IYC', 'IYE',
       'IYG', 'IYH', 'IYJ', 'IYK', 'IYM', 'IYR', 'IYW', 'IYZ', 'JNJ',
       'KO', 'LFC', 'LIN', 'LMT', 'MA', 'MCD', 'MSFT', 'NKE', 'NVDA',
       'NVS', 'PBR', 'PEP', 'PFE', 'PLD', 'PSA', 'PTR', 'PYPL', 'RTX',
       'SHW', 'SNP', 'SO', 'SRE', 'T', 'TM', 'TSLA', 'TSM', 'UNP', 'UPS',
       'V', 'WMT', 'DIA', 'QQQ', 'SPY', 'XNTK'], dtype=object)

In [5]:
# Removing the XNTK ticker
data = data[~data.Ticker.isin(['XNTK'])]

In [6]:
data.Ticker.unique()

array(['AAPL', 'ABBV', 'ABT', 'AEP', 'AMT', 'APD', 'BA', 'BABA', 'BAC',
       'BHP', 'BP', 'CCI', 'CHL', 'COST', 'CSGP', 'D', 'DIS', 'ECL',
       'ENB', 'EXC', 'FB', 'FMX', 'GOOG', 'IDU', 'INTC', 'IYC', 'IYE',
       'IYG', 'IYH', 'IYJ', 'IYK', 'IYM', 'IYR', 'IYW', 'IYZ', 'JNJ',
       'KO', 'LFC', 'LIN', 'LMT', 'MA', 'MCD', 'MSFT', 'NKE', 'NVDA',
       'NVS', 'PBR', 'PEP', 'PFE', 'PLD', 'PSA', 'PTR', 'PYPL', 'RTX',
       'SHW', 'SNP', 'SO', 'SRE', 'T', 'TM', 'TSLA', 'TSM', 'UNP', 'UPS',
       'V', 'WMT', 'DIA', 'QQQ', 'SPY'], dtype=object)

In [7]:
# Removing the XNTK ticker
data = data[~data.Ticker.isin(['XNTK'])]

etfs = ['IYH','IYM','IYK','IYJ','IYG','IYW','IYC','IYR','IDU','IYZ','IYE','IYF','SPY','DIA','QQQ']

# Extracting the sector ETFs to a separate variable
sectorETFS = data[data.Ticker.isin(etfs)]

# Removing the ETFs
data = data[~data.Ticker.isin(etfs)]

In [8]:
data.columns

Index(['open', 'high', 'low', 'close', 'spread_open', 'spread_high',
       'spread_low', 'spread_close', 'bidsize_open', 'bidsize_high',
       'bidsize_low', 'bidsize_close', 'ofrsize_open', 'ofrsize_high',
       'ofrsize_low', 'ofrsize_close', 'Ticker', 'sector'],
      dtype='object')

In [9]:
data

open     high      low    close  spread_open  spread_high  \
20200501 0    286.250  289.260  285.870  289.260         0.50         0.50   
         1    289.260  289.350  288.365  289.020         0.24         0.45   
         2    289.035  289.705  288.280  288.580         0.07         0.49   
         3    288.485  289.315  288.280  289.095         0.49         0.49   
         4    289.100  290.435  288.940  290.320         0.16         0.33   
...               ...      ...      ...      ...          ...          ...   
20200529 385  123.950  124.110  123.910  124.100         0.02         0.07   
         386  124.085  124.085  123.920  123.995         0.01         0.06   
         387  123.995  124.355  123.985  124.335         0.01         0.07   
         388  124.335  124.355  124.060  124.075         0.05         0.12   
         389  124.075  124.225  122.810  123.855         0.01         2.43   

              spread_low  spread_close  bidsize_open  bidsize_high  \
20200501 0          0.01          0.24           6.0          95.0   
         1          0.01          0.10           9.0          20.0   
         2          0.01          0.30           1.0          50.0   
         3          0.01          0.17           1.0          25.0   
         4          0.01          0.10          13.0          71.0   
...                  ...           ...           ...           ...   
20200529 385        0.01          0.04           1.0          11.0   
         386        0.01          0.01           1.0           8.0   
         387        0.01          0.05           4.0          16.0   
         388        0.01          0.01           3.0           6.0   
         389        0.01          0.21           1.0          20.0   

              bidsize_low  bidsize_close  ofrsize_open  ofrsize_high  \
20200501 0            1.0           10.0           1.0          85.0   
         1            1.0            1.0           4.0          56.0   
         2            1.0            1.0           1.0          13.0   
         3            1.0           16.0           1.0           8.0   
         4            1.0            1.0           1.0         236.0   
...                   ...            ...           ...           ...   
20200529 385          1.0            1.0           5.0           9.0   
         386          1.0            3.0           1.0           9.0   
         387          1.0            2.0           2.0          10.0   
         388          1.0            2.0           2.0          10.0   
         389          1.0            2.0           4.0          12.0   

              ofrsize_low  ofrsize_close Ticker              sector  
20200501 0            1.0            4.0   AAPL          Technology  
         1            1.0            1.0   AAPL          Technology  
         2            1.0            1.0   AAPL          Technology  
         3            1.0            1.0   AAPL          Technology  
         4            1.0            1.0   AAPL          Technology  
...                   ...            ...    ...                 ...  
20200529 385          1.0            1.0    WMT  Consumer Defensive  
         386          1.0            2.0    WMT  Consumer Defensive  
         387          1.0            2.0    WMT  Consumer Defensive  
         388          1.0            4.0    WMT  Consumer Defensive  
         389          1.0            1.0    WMT  Consumer Defensive  

[429000 rows x 18 columns]

In [10]:
########### Generate Features ################

n_feature_lags = 1

# features = generateFeatures_multi_final(data = data, 
#                                   listOfFeatures = [
#                                                     'pastobs',
#                                                     'spread',
#                                                     'bidsize',
#                                                     'ofrsize',
# #                                                     'stok',
# #                                                     'stod',
# #                                                     'sstod',
# #                                                     'wilr',
# #                                                     'roc',
# #                                                     'rsi',
# #                                                     'atr',
# #                                                     'cci',
# #                                                     'dpo',
# #                                                     'sma',
# #                                                     'ema',
# #                                                     'macd',
# #                                                       'macd_diff',
# #                                                       'macd_signal',
# #                                                     'dis5',
# #                                                     'dis10',
#                                                       'sector'
#                                                    ], 
#                                    feature_lags = n_feature_lags
#                                      ,stockTable=stockTable)
features = generateFeatures_multi_final(data = data, 
                                  listOfFeatures = [
                                                    'pastobs',
                                                    'spread',
                                                    'bidsize',
                                                    'ofrsize',
                                                    'stok',
                                                    'stod',
                                                    'sstod',
#                                                     'wilr',
                                                    'roc',
                                                    'rsi',
                                                    'atr',
                                                    'cci',
                                                    'dpo',
                                                    'sma',
                                                    'ema',
                                                    'macd',
                                                      'macd_diff',
                                                      'macd_signal',
                                                    'dis5',
                                                    'dis10',
                                                      'sector'
                                                   ], 
                                   feature_lags = n_feature_lags
                                     ,sectorETFS=sectorETFS)

########### Generate Labels ################

n_classes = 2
# extract first 4 columns as the lag0 or raw OHLC prices (used for labelling)
price_candles = data[['open','high','low','close','Ticker']]

########### Align Data ################

# from imported function (see testing_preprocessing_features_and_labels.ipynb for thorough experimenting with all the cut-offs):    
X, y,indices = align_features_and_labels_multi_final(price_candles = price_candles, 
                                                 all_features = features,
                                                 prediction_horizon = 1, 
                                                 n_feature_lags = n_feature_lags, 
                                                 n_classes = n_classes, # 5,
                                                 safe_burn_in = False, 
                                                 data_sample = 'full',
                                                 splitType='global',
                                                 noise=False,ticker_dummies=False)

INFO:MainThread:numexpr.utils:NumExpr defaulting to 4 threads.
C:\Users\PC\Anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\PC\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


AAPL done
ABBV done
ABT done
AEP done
AMT done
APD done
BA done
BABA done
BAC done
BHP done
BP done
CCI done
CHL done
COST done
CSGP done
D done
DIS done
ECL done
ENB done
EXC done
FB done
FMX done
GOOG done
INTC done
JNJ done
KO done
LFC done
LIN done
LMT done
MA done
MCD done
MSFT done
NKE done
NVDA done
NVS done
Number of NaNs in label: 1. 1 is expected
Returns that lead to NaNs in label: [0.0907158]
PBR done
PEP done
PFE done
PLD done
PSA done
PTR done
PYPL done
RTX done
SHW done
SNP done
SO done
SRE done
T done
TM done
TSLA done
TSM done
UNP done
UPS done
V done
WMT done


## Splitting the data

## Adding ticker dummies

In [14]:
## Adding ticker dummies
tickers = X.pop('ticker')
X = pd.concat([X, pd.get_dummies(tickers, prefix='ticker', drop_first=False)], axis=1)

In [15]:
X.columns

Index(['open_lag0', 'high_lag0', 'low_lag0', 'close_lag0', 'spread_open_lag0',
       'spread_high_lag0', 'spread_low_lag0', 'spread_close_lag0',
       'bidsize_open_lag0', 'bidsize_high_lag0',
       ...
       'ticker_SO', 'ticker_SRE', 'ticker_T', 'ticker_TM', 'ticker_TSLA',
       'ticker_TSM', 'ticker_UNP', 'ticker_UPS', 'ticker_V', 'ticker_WMT'],
      dtype='object', length=156)

## Constructing our final train/validation sets

In [16]:
# train_ds = pd.concat([X.iloc[start:end, :] for (start, end) in train_ranges]).reset_index(drop=True)
# train_y = pd.concat([y.iloc[start:end] for (start, end) in train_ranges]).reset_index(drop=True)

# validate_ds = pd.concat([X.iloc[start:end, :] for (start, end) in val_ranges]).reset_index(drop=True)
# val_y = pd.concat([y.iloc[start:end] for (start, end) in val_ranges]).reset_index(drop=True)

# train_ds.shape, train_y.shape, validate_ds.shape, val_y.shape, train_y.shape[0] + val_y.shape[0]

# Let's have a proper split (along tickers & dates)
train_size = 0.8

# Sort the indices
tempIndices = indices.sort_values(['days','timestamps','ticker'])

# Sorting the data
X = X.loc[tempIndices.index,:]#.head(66)
y = y.loc[tempIndices.index,:]

# extracting the first date for the validation data.
first_val_day = int(np.floor(indices.days.unique().shape[0]*0.8))

# Splitting the data
X_train = X[tempIndices.days<tempIndices.days.unique()[first_val_day]].reset_index(drop=True)
y_train = y[tempIndices.days<tempIndices.days.unique()[first_val_day]].reset_index(drop=True)

X_test = X[tempIndices.days>=tempIndices.days.unique()[first_val_day]].reset_index(drop=True)
y_test = y[tempIndices.days>=tempIndices.days.unique()[first_val_day]].reset_index(drop=True)


print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, sep='\n')

(341385, 156)
(341385, 1)
(85690, 156)
(85690, 1)


In [17]:
X_train

,open_lag0,high_lag0,low_lag0,close_lag0,spread_open_lag0,spread_high_lag0,spread_low_lag0,spread_close_lag0,bidsize_open_lag0,bidsize_high_lag0,...,ticker_SO,ticker_SRE,ticker_T,ticker_TM,ticker_TSLA,ticker_TSM,ticker_UNP,ticker_UPS,ticker_V,ticker_WMT
0,-0.545,0.205,-0.635,296.440,0.09,0.20,0.01,0.06,1.0,12.0,...,0,0,0,0,0,0,0,0,0,0
1,-0.200,0.040,-0.255,82.805,0.21,0.26,0.01,0.07,1.0,8.0,...,0,0,0,0,0,0,0,0,0,0
2,0.000,0.030,-0.040,90.855,0.07,0.10,0.01,0.09,1.0,4.0,...,0,0,0,0,0,0,0,0,0,0
3,0.100,0.140,-0.110,81.510,0.06,0.25,0.03,0.16,1.0,9.0,...,0,0,0,0,0,0,0,0,0,0
4,0.010,0.070,-0.045,234.230,0.26,0.30,0.11,0.18,1.0,3.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341380,0.035,0.165,-0.040,51.300,0.17,0.26,0.01,0.02,5.0,23.0,...,0,0,0,0,0,1,0,0,0,0
341381,0.890,0.965,-0.290,169.435,0.35,1.41,0.01,0.93,1.0,10.0,...,0,0,0,0,0,0,1,0,0,0
341382,0.105,0.235,-0.465,97.955,0.86,1.32,0.06,0.15,1.0,6.0,...,0,0,0,0,0,0,0,1,0,0
341383,-0.080,0.540,-0.245,195.705,0.75,1.08,0.01,0.07,1.0,4.0,...,0,0,0,0,0,0,0,0,1,0


In [21]:
np.where((np.sum(np.isinf(X_train.values), axis=1) == 0) == False),
np.where((np.sum(np.isnan(X_train.values), axis=1) == 0) == False)#X_train


(array([], dtype=int64),)

## Pre-processing

In [ ]:
{i:colname for i,colname in enumerate(train_ds.columns)}

In [22]:
# Creating one ppdict for individual preprocessings
# ppdict1 = {'open':'minmax',
#           'high':'log',
#           'low':'log',
#           'close':'std'}
# splitpoint = 32

# # Standardize some features
# ppdict1 = {i:'std' for i in train_ds.columns[0:splitpoint]} 
# # Keep some in actual levels (Dummies in this case).
# ppdict2 = {i:'act' for i in train_ds.columns[splitpoint:]}

pre_procesing_applied = 'std'

# Merging the two
# ppdict = {**ppdict1,**ppdict2}

if  pre_procesing_applied == 'None':
    # do nothing here
    pass

elif  pre_procesing_applied == 'std':

    # splitpoint = int((args.feature-lags+1)*16)#32
    # columns_to_pre_process = [col for col in X_train.columns if 'd_' != col[0:2]]

    # Standardize some features
    ppdict1 = {i:'std' for i in X_train.columns if 'd_' != i[0:2]} 
    # Keep some in actual levels (Dummies in this case).
    ppdict2 = {i:'act' for i in X_train.columns if 'd_' == i[0:2]} 

    # Merging the two
    ppdict = {**ppdict1,**ppdict2}

    # x_train,x_test = pre_processing(x_train,x_test,pp_dict)

elif pre_procesing_applied == 'minmax':

    # splitpoint = int((args.feature-lags+1)*16)#32
    # columns_to_pre_process = [col for col in X_train.columns if 'd_' != col[0:2]]

    # Standardize some features
    ppdict1 = {i:'minmax' for i in X_train.columns if 'd_' != i[0:2]} 
    # Keep some in actual levels (Dummies in this case).
    ppdict2 = {i:'act' for i in X_train.columns if 'd_' == i[0:2]} 

    # Merging the two
    ppdict = {**ppdict1,**ppdict2}

    # x_train,x_test = pre_processing(X_train,X_test,pp_dict)

elif pre_procesing_applied == 'pow':

    # splitpoint = int((args.feature-lags+1)*16)#32
    # columns_to_pre_process = [col for col in X_train.columns if 'd_' != col[0:2]]

    # Standardize some features
    ppdict1 = {i:'pow' for i in X_train.columns if 'd_' != i[0:2]} 
    # Keep some in actual levels (Dummies in this case).
    ppdict2 = {i:'act' for i in X_train.columns if 'd_' == i[0:2]} 

    # Merging the two
    ppdict = {**ppdict1,**ppdict2}

    # x_train,x_test = pre_processing(x_train,x_test,pp_dict)

elif pre_procesing_applied == 'quantgau':

    # splitpoint = int((args.feature-lags+1)*16)#32
    # columns_to_pre_process = [col for col in X_train.columns if 'd_' != col[0:2]]

    # Standardize some features
    ppdict1 = {i:'quantgau' for i in X_train.columns if 'd_' != i[0:2]} 
    # Keep some in actual levels (Dummies in this case).
    ppdict2 = {i:'act' for i in X_train.columns if 'd_' == i[0:2]} 

    # Merging the two
    ppdict = {**ppdict1,**ppdict2}

    # x_train,x_test = pre_processing(x_train,x_test,pp_dict)

elif pre_procesing_applied == 'individual':

    # splitpoint = int((args.feature-lags+1)*16)#32
    # columns_to_pre_process = [col for col in X_train.columns if 'd_' != col[0:2]]

    # Standardize some features
    
    # ppdict1 = {i:'power' for i in X_train.columns if 'd_' != i[0:2]}


    # Keep some in actual levels (Dummies in this case).
    ppdict2 = {i:'act' for i in X_train.columns if 'd_' == i[0:2]} 

    # Merging the two
    ppdict = {**ppdict1,**ppdict2}

    # x_train,x_test = pre_processing(x_train,x_test,pp_dict)

elif pre_procesing_applied == 'stacked':

    # splitpoint = int((args.feature-lags+1)*16)#32
    # columns_to_pre_process = [col for col in X_train.columns if 'd_' != col[0:2]]

    # Standardize some features
    
    for j in ['pow','std','minmax']:

        ppdict1 = {i:j for i in X_train.columns if 'd_' != i[0:2]}

        # Keep some in actual levels (Dummies in this case).
        ppdict2 = {i:'act' for i in X_train.columns if 'd_' == i[0:2]} 

        # Merging the two
        ppdict = {**ppdict1,**ppdict2}

        X_train,X_test = pre_processing(X_train,X_test,ppdict)

if pre_procesing_applied not in ['None','stacked']:
    X_train,X_test = pre_processing(X_train,X_test,ppdict)

In [53]:
ppX_train.iloc[:,0].mean(),ppX_train.iloc[:,0].std()

(-1.8927265537610815e-16, 1.000001457346533)

## Prepping for models

In [8]:
N_VALIDATION = val_y.shape[0] #int(1e3)
N_TRAIN = train_y.shape[0] #int(1e4)
# BUFFER_SIZE = int(1e4)
BATCH_SIZE = 256 #512 #32
MAX_EPOCHS = 500

STEPS_PER_EPOCH = N_TRAIN//BATCH_SIZE

N_REPEAT = int(N_TRAIN / ((STEPS_PER_EPOCH * MAX_EPOCHS) / BATCH_SIZE))
FEATURES = X.shape[1]

N_TRAIN, N_VALIDATION, N_TRAIN + N_VALIDATION, STEPS_PER_EPOCH, N_REPEAT, STEPS_PER_EPOCH * MAX_EPOCHS

(343090, 85800, 428890, 1340, 131, 670000)

## A Logistic Regression model in TF/Keras

In [55]:
METRICS = [
      #keras.metrics.TruePositives(name='tp'),
      #keras.metrics.FalsePositives(name='fp'),
      #keras.metrics.TrueNegatives(name='tn'),
      #keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      #keras.metrics.Precision(name='precision'),
      #keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]

# def make_model(metrics = METRICS, output_bias=None):
#   if output_bias is not None:
#     output_bias = tf.keras.initializers.Constant(output_bias)
#   model = keras.Sequential([
#       keras.layers.Dense(
#           16, activation='relu',
#           input_shape=(train_features.shape[-1],)),
#       keras.layers.Dropout(0.5),
#       keras.layers.Dense(1, activation='sigmoid',
#                          bias_initializer=output_bias),
#   ])

#   model.compile(
#       optimizer=keras.optimizers.Adam(lr=1e-3),
#       loss=keras.losses.BinaryCrossentropy(),
#       metrics=metrics)

#   return model

# model = keras.Sequential({
#   keras.layers.Dense(1, input_shape=(FEATURES,))
# })

model = keras.Sequential([
#     keras.layers.Flatten(input_shape=(28, 28)),
#     keras.layers.Dense(128, activation='relu'),
#     keras.layers.Dense(10)
    keras.layers.Dense(1,
                       input_shape=(FEATURES,),
                       activation='sigmoid',
                       kernel_regularizer=regularizers.l2(1))
])

model.summary()

# with final activation (Keras/TF tutorial advises against this practice, but they also use it later in the tutorial)
# model = keras.Sequential({
#   keras.layers.Dense(1, input_shape=(FEATURES,), activation='sigmoid')
# })

#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy', ])
model.compile(
              optimizer=keras.optimizers.Adam(), #lr=1e-3
              loss=keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=METRICS)

early_stopping = tf.keras.callbacks.EarlyStopping(
                                                monitor='val_auc', 
                                                verbose=1,
                                                patience=100,
                                                mode='max',
                                                restore_best_weights=True)

def get_callbacks(run_id):
      return [
             tfdocs.modeling.EpochDots(),
             early_stopping,
             tf.keras.callbacks.TensorBoard(logdir), #/run_id),
      ]

baseline_history = model.fit(
                            train_ds, #train_features,
                            train_y, #train_labels,
                            batch_size=512, #BATCH_SIZE,
                            epochs=1000, #EPOCHS,
                            callbacks = get_callbacks(run_id = 'first'), #[early_stopping],
                            validation_data=(validate_ds, val_y),
                            verbose=0) #(val_features, val_labels))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 99        
Total params: 99
Trainable params: 99
Non-trainable params: 0
_________________________________________________________________



Epoch: 0, accuracy:0.5352,  auc:0.5034,  loss:0.8996,  val_accuracy:0.5456,  val_auc:0.5453,  val_loss:0.6876,  
....................................................................................................
Epoch: 100, accuracy:0.5480,  auc:0.5440,  loss:0.6873,  val_accuracy:0.5454,  val_auc:0.5459,  val_loss:0.6879,  
..................Restoring model weights from the end of the best epoch.
Epoch 00118: early stopping


In [56]:
model.evaluate(validate_ds,  val_y, verbose=2)

2682/2682 - 6s - loss: 0.6879 - accuracy: 0.5457 - auc: 0.5513


[0.6878659725189209, 0.5456876754760742, 0.5513222217559814]

In [11]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [12]:
import datetime
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
%tensorboard --logdir logs

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 9296.